# Light Rail Accessability

This script evaulates the accessability of potential light rail stops within the Canberra-Queanbeyan urban area. It generates data on the amount of population within a specified region of each stop. It does this across three different transport modes: Walking, Cycling and Public Transport.

It takes input point data for the number of stops, and runs based on ABS 2021 Census data at the mesh block level. The ABS datasets are contained with MainDatasets.gdb, and the stops to run on are contained within this geodatabase. It could also be used on any other point input data, to find the population within a distance of that facility.

This tool can be run to find the sum of the following ABS data:
- 2021 Population data
- 2016 Population data
- 2016 Employment data


It outputs a .csv file containing the population for each stop, for each mode. (Pop_LR_Stages_Access.csv)

It uses the Closest Facility Network Analyst Tool, preventing double counting population for each stop. 

In [1]:
import pandas as pd
import numpy as np

import arcpy
from arcgis import GeoAccessor

### Specify the input data to consider
Use the name of a feature class contained within this geodatabase, or otherwise specify the path to that dataset (paths not currently implemented, so don't do this)

In [2]:
#for correct naming follow the convention "LRX_Stops__..." where each X is each stage contained within it e.g. LR3K_Stops
LR_Stops = ["LR1_Stops"]

## Main Code
Run functions below first

In [8]:
LR_Stops = ["LR1_Stops", "LR2_Stops", "LR3Qa_Stops"]
sdf = find_all_stats(LR_Stops, "Walking", Threshold = 10)

In [20]:
sdf

,Stage,FacilityID,Stop Name,Population Sum (2016),Stage Population Sum (2016),Employment Sum (2016),Stage Employment Sum (2016),Population Sum (2021),Stage Population Sum (2021)
0,1,1,Gungahlin Place,1963,27464,2638,28562,3120,32729
1,1,2,Manning Clark North,2799,27464,0,28562,2934,32729
2,1,3,Mapleton Avenue,3487,27464,615,28562,4314,32729
3,1,4,Nullarbor Avenue,2444,27464,352,28562,2984,32729
4,1,5,Well Station Drive,952,27464,0,28562,962,32729
5,1,6,Sandford Street,0,27464,1566,28562,0,32729
6,1,7,EPIC and Racecourse,0,27464,1346,28562,0,32729
7,1,8,Phillip Avenue,1172,27464,0,28562,1252,32729
8,1,9,Swinden Street,1130,27464,211,28562,1232,32729
9,1,10,Dickson Interchange,1540,27464,1950,28562,2460,32729


In [22]:
## use this for modelling all scenarios

#create the scenarios we want to model
LR_Scenarios = {"Stage 1": ["LR1_Stops"],
                "Stage 1 + 2": ["LR1_Stops", "LR2_Stops"],
                "Stage 1 + 3 (Queanbeyan)": ["LR1_Stops", "LR3Q_Stops"],
                "Stage 1, 2 + Kingston Extension": ["LR1_Stops", "LR2_Stops", "LR3Ka_Stops"],
                "Stage 1, 2 + Constitution Ave": ["LR1_Stops", "LR2_Stops", "LR3C_Stops"],
                "Stage 1, 2 + 3 (Kingston)": ["LR1_Stops", "LR2_Stops", "LR3K_Stops"],
                "Stage 1, 2 + 3 (Queanbeyan)": ["LR1_Stops", "LR2_Stops", "LR3Q_Stops"],
                "Stage 1, 2, 3 (Qbyn), + 4 (Belc)": ["LR1_Stops", "LR2_Stops", "LR3Q_Stops", "LR4_Stops"]
               }


#specify mode and threshold
Threshold = 10
Mode = "Walking"

#specify path to store in
path = "C:/Users/gjames/Documents/Honours Data/Light Rail Results/"


#find the data for each of the scenarios and add them to sheet in the .xlsx file
scenarios_dict = {}
for LR_Scenario in LR_Scenarios.keys():
    LR_Stops = LR_Scenarios[LR_Scenario]
    
    scenario_stats_df = find_all_stats(LR_Stops, Mode, Threshold = Threshold)
    
    #store for exporting to excel
    scenarios_dict[LR_Scenario] = scenario_stats_df

#LR_Scenarios = {"Stage 1, 2 + 4 (Belconnen)": ["LR1_Stops", "LR2_Stops"]}

#create an excel file to store them in
with pd.ExcelWriter(path + 'Light_Rail_' + str(Threshold) + 'min_' + Mode + '_Accessability1.xlsx') as writer:
    for LR_Scenario in LR_Scenarios.keys():
        scenarios_dict[LR_Scenario].to_excel(writer, sheet_name = LR_Scenario, index = False)
    

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [65]:

#specify mode and threshold
# Threshold = 10
# Mode = "Walking"

# #specify path to store in
# path = "C:/Users/gjames/Documents/Honours Data/Light Rail Results/"

# #create an excel file to store them in
# writer = pd.ExcelWriter(path + 'Light_Rail_' + str(Threshold) + 'min_' + Mode + '_Accessability.xlsx') 

# df = pd.DataFrame([[1,2],[3,4]])
# with pd.ExcelWriter(path + 'Light_Rail_' + str(Threshold) + 'min_' + Mode + '_Accessability.xlsx') as writer:
#     df.to_excel(writer, sheet_name = "h", index = False)

## Function defintions

In [4]:
def find_sum_isochrone(LR_Stops, Mode, Field = "2021 Population", Threshold = 10):
    '''
    Generates data on the amount of population within a specified access time of each stop for a given mode. 
    Uses the Closest Facility Tool.
    
    Inputs: LR_Stops - a list of point data feature classess containing the stops to find the population around. 
                example: ["LR1_Stops", "LR2_Stops", "LR3K_Stops"]
                These must have the fields "stage" and "stopname"
    
            Mode - The mode to be used in the analysis ("Walking", "Cycling" or "Public transit time")
                The network dataset to be used is set based of the mode as one of:
                    "C:/Users/gjames/Documents/ArcGIS/Projects/Paths_NetworkDataset/Paths_NetworkDataset.gdb/Paths_ND/Paths_ND"
                    "C:/Users/gjames/Documents/ArcGIS/Projects/GTFS_Integrated/GTFS_Integrated.gdb/GTFS_CQ/TransitNetwork_ND"
            
            Field - the find to sum up for each input point
                This can be one of: 
                    "2021 Population"
                    "2016 Population"
                    "2016 MB_Employment_Count"
            
            Threshold - the access time to sum within, in minutes
            
    Outputs: A .csv file for each of the input layers, in the Honours/Data/OD Data Production folder
    '''
    
    
    arcpy.env.overwriteOutput = True #always important to include - allows tool to be run more than once without creating extra layers
    
    
    #changed for easier understanding of input
    if Field == "2016 Employment":
        Field = "2016 MB_Employment_Count"
    
    #initialise the network analysis layer
    
    #give it a name corresponding to this run
    CF_layer_name = "CF_LR"
    for layer in LR_Stops:
        CF_layer_name += layer.split("_")[0][2:] + "_"

    if Mode == "Walking" or Mode == "Cycling":
        Network_Dataset = "C:/Users/gjames/Documents/ArcGIS/Projects/Paths_NetworkDataset/Paths_NetworkDataset.gdb/Paths_ND/Paths_ND"
        
        CF_layer_name += Mode
        arcpy.na.MakeClosestFacilityAnalysisLayer(Network_Dataset, CF_layer_name, Mode, cutoff = Threshold) 
        
        
    elif Mode == "Public transit time":
        Network_Dataset = "C:/Users/gjames/Documents/ArcGIS/Projects/GTFS_Integrated/GTFS_Integrated.gdb/GTFS_CQ/TransitNetwork_ND"
        
        CF_layer_name+="PT"
        # this is done separately as adding a time of day to Paths_ND signficantly adds computation time
        arcpy.na.MakeClosestFacilityAnalysisLayer(Network_Dataset, CF_layer_name, Mode, cutoff = Threshold,
                                           time_of_day = '4/1/1900 11:00 AM', line_shape = "ALONG_NETWORK")
        
    else:
        return 'Invalid Mode, choose one of "Walking", "Cycling" or "Public transit time"'
    
    
    
    
    #add population/employment data to the incidents
    
    if Field.split(" ")[1] == "Population":
        if Field.split(" ")[0] == "2021":
            incidents_to_add = "MB21_CQ_with_pop_test_points"
            #replace this when created
            r"C:\Users\gjames\Documents\ArcGIS\Projects\MainDatasets\MainDatasets.gdb\MB21_CQ_pop_points"
        
        elif Field.split(" ")[0] == "2016":
            incidents_to_add = r"C:\Users\gjames\Documents\ArcGIS\Projects\MainDatasets\MainDatasets.gdb\MB16_CQ_pop_points"
            
        else:
             return 'Invalid Field, choose one of "2021 Population", "2016 Population" or "2016 MB_Employment_Count"'
    
    elif Field.split(" ")[1] == "MB_Employment_Count":
        incidents_to_add = r"C:\Users\gjames\Documents\ArcGIS\Projects\MainDatasets\MainDatasets.gdb\Nonzero_MB16_CQ_empl_points"
    
    else:
             return 'Invalid Field, choose one of "2021 Population", "2016 Population" or "2016 MB_Employment_Count"'
    
    #replace this later, added to run faster
    arcpy.na.AddLocations(CF_layer_name, "Incidents", incidents_to_add, append = "CLEAR")
    
    
    
    
    
    #add stops to the facilities
    
    #clear any previous run when adding the first layer
    arcpy.na.AddLocations(CF_layer_name, "Facilities", LR_Stops[0], append = "CLEAR")
    for stops_layer in LR_Stops[1:]:
        arcpy.na.AddLocations(CF_layer_name, "Facilities", stops_layer)
    
    
    
    
    #solve
    arcpy.na.Solve(CF_layer_name, "SKIP", "TERMINATE", None, '')
    
    
    
    #join the population to the Routes
    # The object ID could be OBJECTID or OBJECTID_1
    try:
        arcpy.management.AddJoin(CF_layer_name + "/Routes", "IncidentID", 
                             incidents_to_add, "OBJECTID_1")
    except:
        arcpy.management.AddJoin(CF_layer_name + "/Routes", "IncidentID", 
                             incidents_to_add, "OBJECTID")
    
    
    #convert to a dataframe to do operations, and find the name of the layer to be used (it will be CFRoutesXXXXXXX)
    sdf = pd.DataFrame(GeoAccessor.from_featureclass(CF_layer_name + "/Routes"))
    
    
    #find the name of the routes table, and the name of the incidents data
    CF_routes = sdf.columns[0].split(".")[0] #target name will be at the front of the joined table, and joined name will be at the back
    incidents_data = sdf.columns[-2].split(".")[0] #second last as the field "SHAPE" is last
    
    
    #compute summary statistics
    stats_df = compute_statistics(CF_layer_name, LR_Stops, Field = Field)
    
    return stats_df
    
    #export layer - do outside of code
    

In [5]:
def compute_statistics(CF_layer_name, LR_Stops, Field = "2021 Population"):
    '''
    Creates a dataframe containing the sum of all of the population close to each stop, and the sum of all the 
    populations in a stage.
    
    Inputs: CF_layer_name - The name of the Closest Facility output, containing a Routes layer correctly joined to the population
    
            LR_Stops - the point data used as input facilities
            
            Sum_Field - The field to sum, Person for population, and MB_Employment_Count for Employment
            
    Outputs: A dataframe with the following columns - ["Stage", "FacilityID", "Stop Name", "Population Sum", "Stage Population Sum"]
    '''
    
    
    if Field.split(" ")[1] == "MB_Employment_Count":
        Sum_Field = "MB_Employment_Count"
    elif Field.split(" ")[1] == "Population":
        Sum_Field = "Person"
    
    
    #find the Population Sum and FacilityID fields
    
    #convert to a dataframe to do operations, and find the name of the layer to be used (it will be CFRoutesXXXXXXX)
    sdf = pd.DataFrame(GeoAccessor.from_featureclass(CF_layer_name + "/Routes"))
    
    
    #find the name of the routes table, and the name of the incidents data
    # target name will be at the front of the joined table, and joined name will be at the back
    CF_routes = sdf.columns[0].split(".")[0]
    incidents_data = sdf.columns[-2].split(".")[0] #second last as the field "SHAPE" is last
    
    
    #find the range of all the facilityIDs, and create fields for each of the possible facilities (including no destinations ones)
    FacilityID = [int(i) for i in range(1,max(sdf[CF_routes + ".FacilityID"].unique())+1)]
    
    #find the sum of the population field by FacilityID
    Pop_sum_ser = sdf.groupby(CF_routes + ".FacilityID")[incidents_data + "."+ Sum_Field].sum()
    
    #account for 0 value fields when finding population sums
    Population_Sum = []
    for facility in FacilityID:
        if facility in Pop_sum_ser.keys():
            Population_Sum.append(int(Pop_sum_ser[facility]))
        #account for 0 value fields when finding population sums
        else:
            Population_Sum.append(0)
            
    
    
    #find the Stage and Stop Name field
    
    #go through each set of input points, in the same order as before, and add the corresponding data into the correct lists
    Stage = []
    Stop_Name = []
    for stop_layer in LR_Stops:
        layers_stage = list(GeoAccessor.from_featureclass(stop_layer)["stage"])
        layers_stopnames = list(GeoAccessor.from_featureclass(stop_layer)["stopname"])
        
        Stage += [int(stage) for stage in layers_stage]
        Stop_Name += layers_stopnames
        
    
    
    #find the sum by stage
    stage_sum_df = pd.DataFrame([Stage, Population_Sum]).transpose().rename(columns = {0: "Stage", 1 : "Population_Sum"})
    
    stage_sum_series = stage_sum_df.groupby("Stage")["Population_Sum"].sum()
    
    stage_sum_list = list(stage_sum_df.groupby("Stage")["Population_Sum"].sum())
    
         
    Stage_Population_Sum = []
    
    for stage_num in Stage:
         Stage_Population_Sum.append(int(stage_sum_series[stage_num]))

            
    
    #convert found data into a dataframe to output
    stops_statistics_df = pd.DataFrame([Stage, FacilityID, Stop_Name, Population_Sum, Stage_Population_Sum])
    stops_statistics_df = stops_statistics_df.transpose()
    
    year = " (" + Field.split(" ")[0] + ")"
    stops_statistics_df.columns = ["Stage", "FacilityID", "Stop Name", "Population Sum" + year, "Stage Population Sum" +year]
    
    #if we have employment data, change the names to employment
    if Sum_Field == "MB_Employment_Count":
        stops_statistics_df = stops_statistics_df.rename(
            columns = {"Population Sum" + year : "Employment Sum" + year, 
                       "Stage Population Sum" + year : "Stage Employment Sum" + year})
    
    
    return stops_statistics_df
    

In [6]:
def find_all_stats(LR_Stops, Mode, Threshold = 10):
    '''Finds the sums for 2016 and 2021 population and 2016 Employment, and places these
    in a dataframe'''
    
    #the 2021 population is done last, so that the visual layer contains all possible routes
    
    #create an initial dataframe for the 2021 Population data
    isochrone_df = find_sum_isochrone(LR_Stops, Mode, Field = "2016 Population", Threshold = 10)
    
    #find the other data, and merge it will this dataframe
    for Field in ["2016 MB_Employment_Count", "2021 Population"]:
        
    
        other_isochrones = find_sum_isochrone(LR_Stops, Mode, Field = Field, Threshold = 10)
        
        if Field == "2016 MB_Employment_Count":
            isochrone_df = isochrone_df.merge(other_isochrones[["Employment Sum (2016)", "Stage Employment Sum (2016)"]],
                                         left_index = True, right_index = True)
            
        elif Field == "2021 Population":
            isochrone_df = isochrone_df.merge(other_isochrones[["Population Sum (2021)", "Stage Population Sum (2021)"]],
                                         left_index = True, right_index = True)
            
    
    return isochrone_df

### End of function Section

Testing from here on

In [49]:
LR_Stops = ["LR1_Stops", "LR3Q_Stops"]
find_all_stats(LR_Stops, "Walking", Threshold = 10)

,Stage,FacilityID,Stop Name,Population Sum (2021),Stage Population Sum (2021),Population Sum (2016),Stage Population Sum (2016),Employment Sum (2016),Stage Employment Sum (2016)
0,1,1,Gungahlin Place,3120,32548,1963,27285,2638,27176
1,1,2,Manning Clark North,2934,32548,2799,27285,0,27176
2,1,3,Mapleton Avenue,4314,32548,3487,27285,615,27176
3,1,4,Nullarbor Avenue,2984,32548,2444,27285,352,27176
4,1,5,Well Station Drive,962,32548,952,27285,0,27176
5,1,6,Sandford Street,0,32548,0,27285,1566,27176
6,1,7,EPIC and Racecourse,0,32548,0,27285,1346,27176
7,1,8,Phillip Avenue,1252,32548,1172,27285,0,27176
8,1,9,Swinden Street,1232,32548,1130,27285,211,27176
9,1,10,Dickson Interchange,2460,32548,1540,27285,1950,27176


In [46]:
LR_Stops = ["LR1_Stops", "LR3Q_Stops"]
sdf = find_sum_isochrone(LR_Stops, "Walking", Field = "2016 MB_Employment_Count", Threshold = 10)

In [47]:
sdf#[["Employment Sum (2016)", "Stage Employment Sum (2016)"]]

,Stage,FacilityID,Stop Name,Employment Sum (2016),Stage Employment Sum (2016)
0,1,1,Gungahlin Place,2638,27176
1,1,2,Manning Clark North,0,27176
2,1,3,Mapleton Avenue,615,27176
3,1,4,Nullarbor Avenue,352,27176
4,1,5,Well Station Drive,0,27176
5,1,6,Sandford Street,1566,27176
6,1,7,EPIC and Racecourse,1346,27176
7,1,8,Phillip Avenue,0,27176
8,1,9,Swinden Street,211,27176
9,1,10,Dickson Interchange,1950,27176


In [8]:
CF_layer_name = "CF_LR1_Walking"

In [86]:
arcpy.analysis.Statistics(CF_layer_name + "/Routes", 
                          r"C:\Users\gjames\Documents\ArcGIS\Projects\Light_Rail_Accessability\Light_Rail_Accessability.gdb\Routes_Statisticstest",
                          incidents_data + ".Person SUM", 
                          CF_routes + ".FacilityID")

<Result 'C:\\Users\\gjames\\Documents\\ArcGIS\\Projects\\Light_Rail_Accessability\\Light_Rail_Accessability.gdb\\Routes_Statisticstest'>

In [88]:
#sdf = arcgis.features.SpatialDataFrame.from_featureclass(CF_layer_name + "/Routes")
sdf = pd.DataFrame(GeoAccessor.from_featureclass(CF_layer_name + "/Routes"))

In [72]:
CF_routes = sdf.columns[0].split(".")[0]
incidents_data = sdf.columns[-2].split(".")[0]
print(incidents_data)
CF_routes

MB21_CQ_with_pop_test_points


'CFRoutes16hjhyg'

In [117]:
df2 = pd.DataFrame(sdf.groupby(CF_routes + ".FacilityID")[incidents_data + ".Person"].sum())

In [140]:
list(sdf.groupby(CF_routes + ".FacilityID")[incidents_data + ".Person"].sum().index)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [154]:
list(GeoAccessor.from_featureclass("LR1_Stops")["stopname"])

['Gungahlin Place', 'Manning Clark North', 'Mapleton Avenue', 'Nullarbor Avenue', 'Well Station Drive', ' Sandford Street', 'EPIC and Racecourse', 'Phillip Avenue', 'Swinden Street', 'Dickson Interchange', 'Macarthur Avenue', 'Ipima Street', 'Elouera Street', 'Alinga Street']

In [9]:
Routes_pop = CF_layer_name + "/Routes"
Routes_pop

'CF_LR1_Walking/Routes'

In [144]:
LR_Stops

['LR1_Stops']

In [149]:
compute_statistics(Routes_pop, LR_Stops)

[3120, 2934, 4314, 2984, 962, 0, 0, 1252, 1232, 2460, 3734, 3103, 4270, 3632]

In [152]:
list(sdf[CF_routes + ".FacilityID"])

[11, 13, 13, 12, 13, 12, 14, 12, 12, 12, 13, 13, 12, 12, 12, 12, 12, 12, 12, 13, 12, 11, 12, 12, 11, 11, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 10, 11, 10, 10, 10, 10, 10, 10, 11, 10, 11, 8, 8, 9, 9, 9, 9, 9, 9, 10, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 10, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 3, 3, 11, 10, 10, 10, 10, 10, 10, 10, 11, 11, 10, 10, 10, 10, 9, 11, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 11, 11, 13, 13, 12, 12, 11, 12, 12, 12, 13, 12, 12, 12, 13, 12, 12, 13, 13, 8, 8, 8, 8, 8, 8, 7, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 8, 10, 3, 3, 3, 2, 4, 4, 5, 5, 5, 5, 5, 4, 5, 8, 10, 10, 10, 2, 10, 7, 11, 12, 12, 1, 13, 13, 10, 10, 13, 13, 11, 1, 13, 13, 13, 12, 12, 11, 12, 1, 14, 4, 4, 14, 14, 4, 2, 3, 12, 14, 14, 12, 12, 14, 12, 12, 13, 11, 11, 13, 11, 11, 11, 12, 12, 1, 11, 1, 2

In [192]:
sdf.groupby(CF_routes + ".FacilityID")[incidents_data + ".Person"].sum()

CFRoutes16hjhyg.FacilityID
1     3120
2     2934
3     4314
4     2984
5      962
6        0
7        0
8     1252
9     1232
10    2460
11    3734
12    3103
13    4270
14    3632
Name: MB21_CQ_with_pop_test_points.Person, dtype: int64

In [159]:
Stage = [1,1,1,1,1]

In [162]:
[1]*5

[1, 1, 1, 1, 1]

In [146]:
list(sdf.groupby(CF_routes + ".FacilityID")[incidents_data + ".Person"].sum())

[3120, 2934, 4314, 2984, 962, 0, 0, 1252, 1232, 2460, 3734, 3103, 4270, 3632]

In [118]:
df2.columns[0]

'MB21_CQ_with_pop_test_points.Person'

In [119]:
df2 = df2.rename(columns = {df2.columns[0]: "SUM"})

In [123]:
df2.to_csv("C:/Users/gjames/Documents/Honours Data/Output Data testing/routes_sum.csv", index = False)

In [131]:
list(GeoAccessor.from_featureclass("LR1_Stops")["Stop_No"])

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0]

In [78]:
, columns = ["Stop No", "Population", "Stage Total"]pd.DataFrame(df1)

,MB21_CQ_with_pop_test_points.Person
CFRoutes16hjhyg.FacilityID,
1,3120
2,2934
3,4314
4,2984
5,962
6,0
7,0
8,1252
9,1232


In [61]:
sdf.columns

Index(['CFRoutes16hjhyg.ObjectID', 'CFRoutes16hjhyg.FacilityID',
       'CFRoutes16hjhyg.FacilityRank', 'CFRoutes16hjhyg.Name',
       'CFRoutes16hjhyg.IncidentCurbApproach',
       'CFRoutes16hjhyg.FacilityCurbApproach', 'CFRoutes16hjhyg.IncidentID',
       'CFRoutes16hjhyg.StartTime', 'CFRoutes16hjhyg.EndTime',
       'CFRoutes16hjhyg.StartTimeUTC', 'CFRoutes16hjhyg.EndTimeUTC',
       'CFRoutes16hjhyg.Total_Walk_Time', 'CFRoutes16hjhyg.Total_Length',
       'CFRoutes16hjhyg.Total_Cycle_Time',
       'MB21_CQ_with_pop_test_points.OBJECTID_1',
       'MB21_CQ_with_pop_test_points.MB_CODE21',
       'MB21_CQ_with_pop_test_points.MB_CAT21',
       'MB21_CQ_with_pop_test_points.CHG_FLAG21',
       'MB21_CQ_with_pop_test_points.CHG_LBL21',
       'MB21_CQ_with_pop_test_points.SA1_CODE21',
       'MB21_CQ_with_pop_test_points.SA2_CODE21',
       'MB21_CQ_with_pop_test_points.SA2_NAME21',
       'MB21_CQ_with_pop_test_points.SA3_CODE21',
       'MB21_CQ_with_pop_test_points.SA3_NAME21',
   

In [ ]:
arcpy.na.MakeClosestFacilityAnalysisLayer(r"C:\Users\gjames\Documents\ArcGIS\Projects\Paths_NetworkDataset\Paths_NetworkDataset.gdb\Paths_ND\Paths_ND", 
                                          "Closest Facility", "Walking", "TO_FACILITIES", 
                                          None, 1, None, "LOCAL_TIME_AT_LOCATIONS", "START_TIME", "ALONG_NETWORK", None, "NO_DIRECTIONS", "SKIP")

In [ ]:
arcpy.analysis.Statistics(r"Closest Facility\Routes", 
                          r"C:\Users\gjames\Documents\ArcGIS\Projects\Light_Rail_Accessability\Light_Rail_Accessability.gdb\Routes_Statistics",
                          "MB21_CQ_with_pop_test_points.Person SUM", 
                          "CFRoutesoopgjg.FacilityID")

In [ ]:
arcpy.na.Solve("Closest Facility", "SKIP", "TERMINATE", None, '')

In [8]:
Field = "2021 Population"

In [38]:
arcpy.management.AddJoin(r"Closest Facility\Routes", "IncidentID", "MB21_CQ_with_pop_test_points", "OBJECTID_1")

<Result 'Closest Facility\\Routes'>

In [ ]:
arcpy.na.AddLocations("Closest Facility", "Incidents", "MB21_CQ_with_pop_test_points", "Name # #;TargetFacilityCount # #;CurbApproach # 0;Attr_Walk_Time # 0;Attr_Length # 0;Attr_Cycle_Time # 0;Cutoff_Walk_Time # #;Cutoff_Length # #;Cutoff_Cycle_Time # #", "5000 Meters", None, "Intrg_CQ_roads SHAPE;Intrg_cyclepaths4 SHAPE;Intrg_footpaths SHAPE;Intrg_footpath_connectivity SHAPE;Intrg_resi_streets SHAPE;Paths_ND_Junctions NONE", "MATCH_TO_CLOSEST", "CLEAR", "NO_SNAP", "5 Meters", "EXCLUDE", None)

In [4]:
#for correct naming follow the convention "LRX_X_..._X_Stops__..." where each X is each stage contained within it e.g. LR1_3K_4_Stops
    stage_sum_df = pd.DataFrame([Stage, Population_Sum], columns = ["Stage", "Population_Sum"])
    
    stage_sum_list = list(stage_sum_df.groupby("Stage")["Population_Sum"].sum())
    
    #initialise stage
    
    
        
    stage_num = 'Initialise'
    stage_sum = 0
    stage_index_start = 0
    Stage_Population_Sum = [0]*len(Population_Sum)
    
    for i in range(0, len(Population_Sum)):
        #everytime the stage changes, add the data to the list and reset the sum 
        if stage_num != Stage[i]:
            
            Stage_Population_Sum[stage_index_start:i] = [stage_sum]*(i-stage_index_start)
            
            stage_sum = 0
            
            #the reset index will be the start of the next stage
            stage_index_start = i
            
        stage_sum += Population_Sum[i]
            
        #to check for next cycle
        stage_num = Stage[i]    
        

In [90]:
def arcgis_table_to_df(in_fc, input_fields=None, query=""):
    """Function will convert an arcgis table into a pandas dataframe with an object ID index, and the selected
    input fields using an arcpy.da.SearchCursor.
    :param - in_fc - input feature class or table to convert
    :param - input_fields - fields to input to a da search cursor for retrieval
    :param - query - sql query to grab appropriate values
    :returns - pandas.DataFrame"""
    OIDFieldName = arcpy.Describe(in_fc).OIDFieldName
    if input_fields:
        final_fields = [OIDFieldName] + input_fields
    else:
        final_fields = [field.name for field in arcpy.ListFields(in_fc)]
    data = [row for row in arcpy.da.SearchCursor(in_fc,final_fields,where_clause=query)]
    fc_dataframe = pd.DataFrame(data,columns=final_fields)
    fc_dataframe = fc_dataframe.set_index(OIDFieldName,drop=True)
    return fc_dataframe

In [101]:
df =arcgis_table_to_df(CF_layer_name + "/Routes")

In [102]:
df

,CFRoutes16hjhyg.Shape,CFRoutes16hjhyg.FacilityID,CFRoutes16hjhyg.FacilityRank,CFRoutes16hjhyg.Name,CFRoutes16hjhyg.IncidentCurbApproach,CFRoutes16hjhyg.FacilityCurbApproach,CFRoutes16hjhyg.IncidentID,CFRoutes16hjhyg.StartTime,CFRoutes16hjhyg.EndTime,CFRoutes16hjhyg.StartTimeUTC,CFRoutes16hjhyg.EndTimeUTC,CFRoutes16hjhyg.Total_Walk_Time,CFRoutes16hjhyg.Total_Length,CFRoutes16hjhyg.Total_Cycle_Time,CFRoutes16hjhyg.Shape_Length,MB21_CQ_with_pop_test_points.OBJECTID_1,MB21_CQ_with_pop_test_points.MB_CODE21,MB21_CQ_with_pop_test_points.MB_CAT21,MB21_CQ_with_pop_test_points.CHG_FLAG21,MB21_CQ_with_pop_test_points.CHG_LBL21,MB21_CQ_with_pop_test_points.SA1_CODE21,MB21_CQ_with_pop_test_points.SA2_CODE21,MB21_CQ_with_pop_test_points.SA2_NAME21,MB21_CQ_with_pop_test_points.SA3_CODE21,MB21_CQ_with_pop_test_points.SA3_NAME21,MB21_CQ_with_pop_test_points.SA4_CODE21,MB21_CQ_with_pop_test_points.SA4_NAME21,MB21_CQ_with_pop_test_points.GCC_CODE21,MB21_CQ_with_pop_test_points.GCC_NAME21,MB21_CQ_with_pop_test_points.STE_CODE21,MB21_CQ_with_pop_test_points.STE_NAME21,MB21_CQ_with_pop_test_points.AUS_CODE21,MB21_CQ_with_pop_test_points.AUS_NAME21,MB21_CQ_with_pop_test_points.AREASQKM21,MB21_CQ_with_pop_test_points.LOCI_URI21,MB21_CQ_with_pop_test_points.Shape_Leng,MB21_CQ_with_pop_test_points.OBJECTID,MB21_CQ_with_pop_test_points.MB_CODE_2021,MB21_CQ_with_pop_test_points.MB_CATEGORY_NAME_2021,MB21_CQ_with_pop_test_points.AREA_ALBERS_SQKM,MB21_CQ_with_pop_test_points.Dwelling,MB21_CQ_with_pop_test_points.Person,MB21_CQ_with_pop_test_points.State,MB21_CQ_with_pop_test_points.MBCODE_21,MB21_CQ_with_pop_test_points.Population_density,MB21_CQ_with_pop_test_points.ORIG_FID
CFRoutes16hjhyg.ObjectID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,"(149.13492082656316, -35.260800332863866)",11,1,Location 603 - Location 11,1,1,603,None,None,None,None,6.421679,535.118548,None,0.005787,603,80000810000,Parkland,0,No change,80105105015,801051050,Ainslie,80105,North Canberra,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,0.0706,http://linked.data.gov.au/dataset/asgsed3/MB/8...,0.010428,60071,8.000081e+10,Parkland,0.0706,0,0,8,80000810000,0.000000,603
2,"(149.131711037661, -35.27346524542094)",13,1,Location 869 - Location 13,2,2,869,None,None,None,None,5.217418,399.628607,None,0.004052,869,80003580000,Commercial,0,No change,80105105119,801051051,Braddon,80105,North Canberra,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,0.0352,http://linked.data.gov.au/dataset/asgsed3/MB/8...,0.007989,60337,8.000358e+10,Commercial,0.0352,121,156,8,80003580000,4431.818182,869
3,"(149.1324341914562, -35.27348646846755)",13,1,Location 870 - Location 13,2,2,870,None,None,None,None,6.663482,526.248131,None,0.005414,870,80003600000,Commercial,0,No change,80105105120,801051051,Braddon,80105,North Canberra,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,0.0367,http://linked.data.gov.au/dataset/asgsed3/MB/8...,0.008169,60338,8.000360e+10,Commercial,0.0367,152,199,8,80003600000,5422.343324,870
4,"(149.1324641631695, -35.2656866034947)",12,1,Location 871 - Location 12,1,1,871,None,None,None,None,4.921843,410.137197,None,0.004047,871,80003610000,Parkland,0,No change,80105105112,801051051,Braddon,80105,North Canberra,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,0.0126,http://linked.data.gov.au/dataset/asgsed3/MB/8...,0.004472,60339,8.000361e+10,Parkland,0.0126,0,0,8,80003610000,0.000000,871
5,"(149.1329412394235, -35.27400233884124)",13,1,Location 872 - Location 13,1,2,872,None,None,None,None,8.932863,726.360768,None,0.007455,872,80003630000,Parkland,0,No change,80105105115,801051051,Braddon,80105,North Canberra,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,0.0598,

In [199]:
    
    
#     stage_sum_list = list(stage_sum_df.groupby("Stage")["Population_Sum"].sum())
#     Stage_Population_Sum = []
#     stage_index = 0
#     prev_stage_index = -1 #so the first stage index will be 0
    
#     for i in range(0, len(Population_Sum)):
#         #everytime the stage changes, move to the next stages sum and add that 
#         if prev_stage_num != Stage[i]:
#             stage_index += 1
        
#         Stage_Population_Sum.append(stage_sum_list[stage_index])
        
#         prev_stage_num = Stage[i]
        
        
        
        
        
#     stage_num = 'Initialise'
#     stage_sum = 0
#     stage_index_start = 0
#     Stage_Population_Sum = [0]*len(Population_Sum)
    
#     for i in range(0, len(Population_Sum)):
#         #everytime the stage changes, add the data to the list and reset the sum 
#         if stage_num != Stage[i]:
            
#             Stage_Population_Sum[stage_index_start:i] = [stage_sum]*(i-stage_index_start)
            
#             stage_sum = 0
            
#             #the reset index will be the start of the next stage
#             stage_index_start = i
            
#         stage_sum += Population_Sum[i]
            
#         #to check for next cycle
#         stage_num = Stage[i]    
        

        

In [2]:
proj = arcpy.mp.ArcGISProject('CURRENT')

In [ ]:
# arcpy.management.AddJoin(r"LR Routes\CFRoutes_LR1_2_3Q_4_pop21_walking", 
#                          "FacilityID", 
#                          r"LR Stops\LR1_2_3Q_4_Stops", 
#                          "OBJECTID_1", 
#                          "KEEP_ALL", 
#                          "NO_INDEX_JOIN_FIELDS")

In [7]:
for layer in proj.listMaps()[0].listLayers():
    print(layer.name)

CF_LR1_2_3Qa_Walking
Facilities
Incidents
Point Barriers
Routes
Line Barriers
Polygon Barriers
CF_LR3Qa_Walking
Facilities
Incidents
Point Barriers
Routes
Line Barriers
Polygon Barriers
CF_LR1_2_3Q_4_Walking
Facilities
Incidents
Point Barriers
Routes
Line Barriers
Polygon Barriers
CF_LR1_2_3Q_Walking
Facilities
Incidents
Point Barriers
Routes
Line Barriers
Polygon Barriers
CF_LR1_2_3K_Walking
Facilities
Incidents
Point Barriers
Routes
Line Barriers
Polygon Barriers
CF_LR1_2_3C_Walking
Facilities
Incidents
Point Barriers
Routes
Line Barriers
Polygon Barriers
CF_LR1_2_3Ka_Walking
Facilities
Incidents
Point Barriers
Routes
Line Barriers
Polygon Barriers
CF_LR1_3Q_Walking
Facilities
Incidents
Point Barriers
Routes
Line Barriers
Polygon Barriers
CF_LR1_2_Walking
Facilities
Incidents
Point Barriers
Routes
Line Barriers
Polygon Barriers
CF_LR1_Walking
Facilities
Incidents
Point Barriers
Routes
Line Barriers
Polygon Barriers
CF_LR1_2_4_Walking
Facilities
Incidents
Point Barriers
Routes
Line Ba

### Joining all stop infomation to all route information
Applies a join to all the CF routes layers

In [7]:
#get a list of all the scenarios
scenario_list = []

proj = arcpy.mp.ArcGISProject('CURRENT')
for layer in proj.listMaps()[0].listLayers():
    if layer.name[:9] == "CFRoutes_":
        scenario = "_".join(layer.name.split("_")[1:-2])
        
        if scenario.split("_")[0] != "LRall" and scenario.split("_")[-1] != "pop21":  # the combined results are not here
            scenario_list.append(scenario)
        print("_".join(layer.name.split("_")[1:-2]))

LRall_pop21
LRall_pop21
LRall
LRall_pop21
LRall_pop21
LR1_pop21
LR1_2_pop21
LR1_3Q_pop21
LR1_2_3Ka_pop21
LR1_2_3Qa_pop21
LR1_2_3C_pop21
LR1_2_3K_pop21
LR1_2_3Q_pop21
LR1_2_4_pop21
LR1_2_3Q_4_pop21
LR1_2_3Qa
LR1_2_3Q_4
LR1_2_3Q
LR1_2_3K
LR1_2_3C
LR1_2_3Ka
LR1_3Q
LR1_2
LR1
LR1_2_4
LR1_pop21


In [8]:
scenario_list

['LR1_2_3Qa', 'LR1_2_3Q_4', 'LR1_2_3Q', 'LR1_2_3K', 'LR1_2_3C', 'LR1_2_3Ka', 'LR1_3Q', 'LR1_2', 'LR1', 'LR1_2_4']

In [9]:
#joining each scenario's stop information to its route layer

## done

# for layer in proj.listMaps()[0].listLayers():
#     #find the CF Routes layers
#     if layer.name[:9] == "CFRoutes_":
        
#         #find the corresponding stops layer
#         scenario = "_".join(layer.name.split("_")[1:-2])
#         if scenario in scenario_list:
#             print(scenario)
            
#             #join the information from the stop
#             stop_layer_name = "LR Stops/" + scenario + "_Stops"
#             print(stop_layer_name)
            
#             arcpy.management.AddJoin("LR Routes/" + layer.name, 
#                                      "FacilityID", 
#                                      stop_layer_name, 
#                                      "OBJECTID_1", 
#                                      "KEEP_ALL", 
#                                      "NO_INDEX_JOIN_FIELDS")

In [10]:
#running feature to feature on each output map

# done

# for scenario in scenario_list:
#     routes_name = "LR Routes/CFRoutes_" + scenario + "_pop21_walking"
#     print(routes_name)
    
#     output_routes_name = routes_name.split("/")[-1] + "_stops"
    
#     arcpy.conversion.FeatureClassToFeatureClass(routes_name,
#                                                 r"C:\Users\gjames\Documents\ArcGIS\Projects\Light_Rail_Accessability\Light_Rail_Accessability.gdb",
#                                                 output_routes_name)

In [11]:
#Add scenario names from the routes layer to the stops layers
for scenario in scenario_list:
    
    #get the correspnding scenario name
    routes_name = "CF Routes and stops/CFRoutes_" + scenario + "_pop21_walking_stops"
    
    scenario_name = GeoAccessor.from_featureclass(routes_name)["Scenario_Name"][0]
    print(scenario_name)

    #get the corresponding stop layer name
    stop_layer_name = "LR Stops/" + scenario + "_Stops"
    print(stop_layer_name)
    
    #add the Scenario field
    arcpy.management.CalculateField(
                stop_layer_name,
                "Scenario", scenario,
                "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
    
    #add the Scenario_Name field
    arcpy.management.CalculateField(
                    stop_layer_name,
                    "Scenario_Name", scenario_name,
                    "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
    
    
    
## TO DO: How to figure out to add strings to field here (needes extra brackets I think "'LR1_2_3Qa'")

# can't reference external variables when calculating field - annoying!

Stage 1, 2 and 3 (Queanbeyan Extension)
LR Stops/LR1_2_3Qa_Stops


Traceback (most recent call last):
  File "<expression>", line 1, in <module>
NameError: name 'LR1_2_3Qa' is not defined


ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (CalculateField).


In [18]:
#Add scenario names from the routes layer to the stops layers

# this will be done by creating a dataframe, and adding a column with the information in it, before merging and exporting back
# to a feature layer

for scenario in scenario_list:
    
    #get the stop layer data
    stop_layer_name = "LR Stops/" + scenario + "_Stops"
    stops_sdf = GeoAccessor.from_featureclass(stop_layer_name)
    print(stop_layer_name)
    
    #get the correspnding scenario name
    routes_name = "CF Routes and stops/CFRoutes_" + scenario + "_pop21_walking_stops"
    
    scenario_name = GeoAccessor.from_featureclass(routes_name)["Scenario_Name"][0]
    print(scenario_name)
    
    #add the scenario name to the new dataframe
    stops_sdf["Scenario_Name"] = [scenario_name]*len(stops_sdf)
    
    #merge the dataframes
    if scenario == scenario_list[0]:
        merged_sdf = stops_sdf
    else:
        merged_sdf = merged_sdf.append(stops_sdf, ignore_index = True)
    

#transfer the merged dataframe back to a feature layer
merged_sdf

LR Stops/LR1_2_3Qa_Stops
Stage 1, 2 and 3 (Queanbeyan Extension)
LR Stops/LR1_2_3Q_4_Stops
Stage 1, 2, 3 (Qbyn) and 4 (Belc)
LR Stops/LR1_2_3Q_Stops
Stage 1, 2 and 3 (Queanbeyan)
LR Stops/LR1_2_3K_Stops
Stage 1, 2 and 3 (Kingston)
LR Stops/LR1_2_3C_Stops
Stage 1, 2 and Constitution Avenue
LR Stops/LR1_2_3Ka_Stops
Stage 1, 2 and Kingston Extension
LR Stops/LR1_3Q_Stops
Stage 1 and 3 (Queanbeyan)
LR Stops/LR1_2_Stops
Stage 1 and 2
LR Stops/LR1_Stops
Stage 1
LR Stops/LR1_2_4_Stops
Stage 1, 2 and 4 (Belconnen)


,OBJECTID_1,objectid,stage,stopname,Stop_No,ORIG_FID,Scenario,SHAPE,Scenario_Name
0,1,1.0,1.0,Gungahlin Place,1.0,0.0,None,"{""x"": 149.13507462259577, ""y"": -35.18556565692...","Stage 1, 2 and 3 (Queanbeyan Extension)"
1,2,2.0,1.0,Manning Clark North,2.0,1.0,None,"{""x"": 149.14400220333903, ""y"": -35.18707790112...","Stage 1, 2 and 3 (Queanbeyan Extension)"
2,3,3.0,1.0,Mapleton Avenue,3.0,2.0,None,"{""x"": 149.15090938516005, ""y"": -35.19384117433...","Stage 1, 2 and 3 (Queanbeyan Extension)"
3,4,4.0,1.0,Nullarbor Avenue,4.0,3.0,None,"{""x"": 149.14918726258213, ""y"": -35.20105746033...","Stage 1, 2 and 3 (Queanbeyan Extension)"
4,5,5.0,1.0,Well Station Drive,5.0,4.0,None,"{""x"": 149.1474088208605, ""y"": -35.208726699777...","Stage 1, 2 and 3 (Queanbeyan Extension)"
...,...,...,...,...,...,...,...,...,...
319,31,NaN,4.0,Eardley Street,NaN,NaN,NaN,"{""x"": 149.09011747034504, ""y"": -35.24044538454...","Stage 1, 2 and 4 (Belconnen)"
320,32,NaN,4.0,University of Canberra,NaN,NaN,NaN,"{""x"": 149.0845522874581, ""y"": -35.239746617612...","Stage 1, 2 and 4 (Belconnen)"
321,33,NaN,4.0,Eastern Valley Way,NaN,NaN,NaN,"{""x"": 149.07367187205932, ""y"": -35.23967018962...","Stage 1, 2 and 4 (Belconnen)"
322,34,NaN,4.0,Belconnen Interchange,NaN,NaN,NaN,"{""x"": 149.0680080353426, ""y"": -35.239962642861...","Stage 1, 2 and 4 (Belconnen)"


In [22]:
len(stops_sdf)

35

In [23]:
#merged_sdf.spatial.to_featureclass(location = r"C:\Users\gjames\Documents\ArcGIS\Projects\Light_Rail_Accessability\Light_Rail_Accessability.gdb\LR_all_Stops_sdf")